In [1]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/drive')
#drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install sentencepiece
!pip install tensorflow==2.7.0
!pip install stanza
!pip install transformers
!pip install tensorflow-addons
!pip install nltk
!pip install textacy

     |████████████████████████████████| 4.0 MB 15.8 MB/s 
     |████████████████████████████████| 6.6 MB 94.1 MB/s 
     |████████████████████████████████| 77 kB 7.1 MB/s 
     |████████████████████████████████| 895 kB 88.0 MB/s 
     |████████████████████████████████| 596 kB 88.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 14.4 MB/s 
     |████████████████████████████████| 489.6 MB 18 kB/s 
     |████████████████████████████████| 1.3 MB 76.9 MB/s 
     |████████████████████████████████| 463 kB 81.0 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully

In [3]:
import tensorflow as tf
print(tf.__version__)

2.7.0


In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split
from textacy.datasets.supreme_court import SupremeCourt
import numpy as np
import re
import unicodedata
import nltk
#from transformers import pipeline
from nltk.corpus import stopwords
from tensorflow import keras
from tensorflow.keras.layers import Dense,Dropout, Input, BatchNormalization
from tqdm import tqdm
import pickle
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import matplotlib.pyplot as plt
import itertools
from sklearn.utils import shuffle
from tensorflow.keras import regularizers
#from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig,TFDistilBertModel,DistilBertTokenizer,DistilBertConfig
import pandas as pd
from transformers import AutoTokenizer, TFAutoModel
import numpy as np
import gc
import math
import json
import stanza
from tensorflow.keras import *
import tensorflow as tf
from tensorflow.keras import *
import tensorflow.keras.backend as K
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from transformers import TFRobertaModel,RobertaTokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.initializers import RandomUniform

from numpy.random import seed
import random as python_random
import os
import sys

np.random.seed(1)
python_random.seed(1)
tf.random.set_seed(1)

In [5]:
!cp "/content/drive/My Drive/labels_sc.txt" "./labels_sc.txt"
!cp "/content/drive/My Drive/labels_sc_279.txt" "./labels_sc_279.txt"

In [6]:
sc = SupremeCourt()
print(sc.info)
sc.download()

texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids

issue_codes = list(sc.issue_area_codes.keys()) # 15 labels
print(issue_codes)
issue_codes.sort()
issue_codes = [str(ic) for ic in issue_codes]

labels_index = dict(zip(issue_codes, np.arange(len(issue_codes))))
print(labels_index)
count=0

for record in sc.records():

        count=count+1
        if record[1]['issue'] == None: # some cases have None as an issue
            labels.append(labels_index['-1'])
        else:
            labels.append(labels_index[record[1]['issue'][:-4]])
        
        new_sen=record[0].split("Footnotes")[0]
        new_new_sen=new_sen.split()
        if len(new_new_sen) >= 2048:
          new_new_sen=new_new_sen[1536:2048]
        
        elif len(new_new_sen) < 512:
          new_new_sen=new_new_sen[0:len(new_new_sen)]
        
        else:
          new_new_sen=new_new_sen[-512:]
          
        new_new_sen=' '.join(new_new_sen)

        texts.append(new_new_sen)
   
len_list = [len(ele.split()) for ele in texts]

print(labels)
print(len(labels))

res = 0 if len(len_list) == 0 else (float(sum(len_list)) / len(len_list))
print("Average Length %s" % res) 
print('Found %s texts.' % len(texts))
print('Found %s labels.' % len(labels_index))

temp_file = open("labels_sc.txt", "r")

data = temp_file.read()
label_list = data.split("\n")
print(label_list)
label_list = label_list[0:-1]
print(label_list)
label_list = [int(i) for i in label_list]

    

{'name': 'supreme_court', 'site_url': 'http://caselaw.findlaw.com/court/us-supreme-court', 'description': 'Collection of ~8.4k decisions issued by the U.S. Supreme Court between November 1946 and June 2016.'}


100%|██████████| 111M/111M [00:02<00:00, 39.1MB/s]


[-1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
{'-1': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, '11': 11, '12': 12, '13': 13, '14': 14}
[8, 1, 8, 2, 8, 8, 8, 9, 7, 8, 1, 2, 1, 1, 8, 4, 8, 8, 12, 3, 3, 7, 3, 12, 1, 8, 8, 8, 8, 8, 8, 8, 1, 9, 5, 9, 9, 9, 11, 8, 8, 8, 4, 8, 8, 8, 8, 1, 3, 9, 3, 7, 1, 2, 9, 9, 7, 8, 8, 1, 10, 7, 8, 8, 9, 8, 7, 9, 9, 12, 7, 2, 8, 1, 11, 11, 1, 7, 7, 12, 1, 9, 8, 10, 12, 7, 8, 2, 8, 9, 9, 1, 8, 9, 1, 7, 12, 10, 10, 10, 8, 3, 7, 9, 8, 9, 1, 8, 8, 2, 7, 2, 9, 9, 11, 8, 8, 12, 12, 7, 8, 12, 4, 9, 3, 1, 12, 1, 1, 8, 8, 3, 8, 8, 8, 8, 9, 1, 8, 8, 10, 1, 8, 2, 8, 8, 7, 3, 8, 2, 4, 4, 9, 8, 10, 12, 12, 1, 1, 9, 1, 1, 1, 8, 2, 2, 8, 1, 1, 2, 2, 1, 2, 8, 1, 9, 9, 8, 8, 4, 2, 9, 9, 8, 3, 4, 3, 1, 8, 8, 2, 1, 9, 7, 8, 8, 1, 12, 3, 8, 2, 4, 2, 9, 12, 1, 4, 1, 8, 8, 8, 2, 2, 8, 9, 8, 8, 8, 10, 9, 8, 7, 9, 1, 1, 9, 4, 2, 4, 2, 2, 1, 7, 8, 11, 11, 3, 9, 2, 4, 8, 9, 1, 8, 1, 1, 4, 9, 1, 1, 8, 8, 2, 1, 8, 4, 2, 8, 9, 8, 8, 2, 8

In [7]:
summarized_data = pd.DataFrame(texts,
               columns =['text'])
summarized_data['label'] = label_list
print(summarized_data)

                                                   text  label
0     held valid run afoul of Rev.Stat. 4888 because...      8
1     Mann Act's coverage beyond the congressional i...      1
2     within its scope all pipe lines that would car...      8
3     ignore the plain import of traditional methods...      2
4     the runway sections as they were intermittentl...      8
...                                                 ...    ...
8414  Opinion reported: Ante, p. 88. DECREE 1. It is...     11
8415  325,000. For the purpose of giving effect to t...     10
8416  v. California was controlling and emphasizing ...     10
8417  passage more recently known as "middle pass," ...     11
8418  original jurisdiction. It is similarly clear t...      9

[8419 rows x 2 columns]


In [8]:
def create_model():
    inps = Input(shape = (max_len,), dtype='int64')
    masks= Input(shape = (max_len,), dtype='int64')
    dbert_layer = dbert_model(inps, attention_mask=masks)[0][:,0,:]
    dense_0 = Dense(512,activation='relu',kernel_regularizer=regularizers.l2(0.01))(dbert_layer)
    dropout_0= Dropout(0.5)(dense_0)
    pred = Dense(15, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout_0)
    model = tf.keras.Model(inputs=[inps,masks], outputs=pred)
    print(model.summary())
    return model   

In [9]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
total_accuracy=0
total_weighted_f1=0
total_micro_f1=0
total_weighted_precision=0
total_micro_precision=0
total_weighted_recall=0
total_micro_recall=0

for i in range(5):
  gc.collect()
  tf.keras.backend.clear_session()
  dbert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
  dbert_model = TFBertModel.from_pretrained('bert-base-uncased')
  max_len=512
  sentences=summarized_data['text']
  labels=summarized_data['label']
  len(sentences),len(labels)
  model_0=create_model()
  input_ids=[]
  attention_masks=[]

  for sent in sentences:
    dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
    input_ids.append(dbert_inps['input_ids'])
    attention_masks.append(dbert_inps['attention_mask'])
  input_ids=np.asarray(input_ids)

  attention_masks=np.array(attention_masks)
  labels=np.array(labels)
  train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.1,random_state=42)
  log_dir='dbert_model'

  model_save_path='./drive/MyDrive/Best-512/best-512-1536-2048-'+str(i)+'-15labels.h5'

  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

  optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
  callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,monitor='val_accuracy',mode='max',save_best_only=True,save_weights_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
  model_0.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
  gpu_info = !nvidia-smi
  gpu_info = '\n'.join(gpu_info)
  if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
  else:
    print(gpu_info)
  history=model_0.fit([train_inp,train_mask],train_label,batch_size=8,epochs=5,validation_data=([val_inp,val_mask],val_label),callbacks=callbacks)
  pred_labels=[]

  model_saved= create_model()
  model_saved.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
  model_saved.load_weights('./drive/MyDrive/Best-512/best-512-1536-2048-'+str(i)+'-15labels.h5')

  for i in range(0,len(val_inp)):
    pred=model_saved.predict([val_inp[i].reshape(1,512),val_mask[i].reshape(1,512)])
    pred_label = pred.argmax(axis=1)
    pred_labels.append(pred_label)
  accuracy=accuracy_score(val_label, pred_labels)
  print("Accuracy: "+str(accuracy))
  total_accuracy=total_accuracy+accuracy
  
  weighted_f1=f1_score(val_label,pred_labels, average='weighted')
  print("Weighted F1: "+ str(weighted_f1))
  total_weighted_f1=total_weighted_f1+weighted_f1
  micro_f1=f1_score(val_label,pred_labels, average='micro')
  print("Micro F1: "+ str(micro_f1))
  total_micro_f1=total_micro_f1+micro_f1

  weighted_precision=precision_score(val_label, pred_labels, average='weighted')
  print("Weighted Precision: " + str(weighted_precision))
  total_weighted_precision=total_weighted_precision+weighted_precision
  micro_precision=precision_score(val_label, pred_labels, average='micro')
  print("Micro Precision: " + str(micro_precision))
  total_micro_precision=total_micro_precision+micro_precision

  weighted_recall=recall_score(val_label, pred_labels, average='weighted')
  print("Weighted Recall: " + str(weighted_recall))
  total_weighted_recall=total_weighted_recall+weighted_recall
  micro_recall=recall_score(val_label, pred_labels, average='micro')
  print("Micro Recall: " + str(micro_recall))
  total_micro_recall=total_micro_recall+micro_recall


print("Average Accuracy: "+str(total_accuracy/5))
print("Average Weighted F1: "+str(total_weighted_f1/5))
print("Average Micro F1: "+str(total_micro_f1/5))
print("Average Weighted Precision: "+str(total_weighted_precision/5))
print("Average Micro Precision: "+str(total_micro_precision/5))
print("Average Weighted Recall: "+str(total_weighted_recall/5))
print("Average Micro Recall: "+str(total_micro_recall/5))

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


...

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Thu Apr 28 18:38:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    40W / 300W |  15237MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


948/948 [==============================] - 336s 338ms/step - loss: 7.1408 - accuracy: 0.5711 - val_loss: 6.2268 - val_accuracy: 0.6449
Epoch 2/5
948/948 [==============================] - 320s 338ms/step - loss: 5.5925 - accuracy: 0.7139 - val_loss: 5.1868 - val_accuracy: 0.6758
Epoch 3/5
948/948 [==============================] - 320s 338ms/step - loss: 4.4834 - accuracy: 0.7896 - val_loss: 4.4690 - val_accuracy: 0.6876
Epoch 4/5
948/948 [==============================] - 320s 338ms/step - loss: 3.5747 - accuracy: 0.8536 - val_loss: 3.9792 - val_accuracy: 0.6960
Epoch 5/5
948/948 [==============================] - 317s 334ms/step - loss: 2.8167 - accuracy: 0.9062 - val_loss: 3.4804 - val_accuracy: 0.6793
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 512)]        0           []   

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
